Helper functions and installing libraries

In [1]:
import requests
import pandas as pd

In [24]:
def app_dictionary(id, list, dictionary):
    if id not in dictionary: #checks if column with key already exists 
        dictionary[id] = [list]
    else:
        dictionary[id].append(list) #if yes, it appends to key value


def run_api(api_call, pagination):
    result = requests.get(api_call)
    result = result.json()
    if pagination == True:
        has_more = result['pagination']['has_more']
    else:
        has_more = False

    return result, has_more

Get league names

In [3]:
leagues = requests.get(f"https://api.sportmonks.com/v3/football/leagues?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep")

leagues = leagues.json()

leagues

{'data': [{'id': 8,
   'sport_id': 1,
   'country_id': 462,
   'name': 'Premier League',
   'active': True,
   'short_code': 'UK PL',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/8/8.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-06 15:30:00',
   'category': 1,
   'has_jerseys': False},
  {'id': 72,
   'sport_id': 1,
   'country_id': 38,
   'name': 'Eredivisie',
   'active': True,
   'short_code': 'NLD E',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/72.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-06 18:00:00',
   'category': 1,
   'has_jerseys': False},
  {'id': 82,
   'sport_id': 1,
   'country_id': 11,
   'name': 'Bundesliga',
   'active': True,
   'short_code': 'GER BI',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/18/82.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-06 17:30:00',
   'category': 1,
   'has

In [4]:
wanted_leagues = ['Premier League', 'La Liga', 'Bundesliga', 'Serie A']
wanted_features = ['id', 'country_id', 'name', 'image_path']

leagues_df = {}

for league in leagues['data']:
    if league['name'] in wanted_leagues:
        for feature in wanted_features:
            app_dictionary(feature, league[f'{feature}'], leagues_df)
        
leagues_df = pd.DataFrame(leagues_df)
leagues_df

,id,country_id,name,image_path
0,8,462,Premier League,https://cdn.sportmonks.com/images/soccer/leagu...
1,82,11,Bundesliga,https://cdn.sportmonks.com/images/soccer/leagu...
2,384,251,Serie A,https://cdn.sportmonks.com/images/soccer/leagu...
3,564,32,La Liga,https://cdn.sportmonks.com/images/soccer/leagu...
4,648,5,Serie A,https://cdn.sportmonks.com/images/soccer/leagu...


Get season ID by search

In [21]:
season_names = ['2023']
season_data = {'data':[]}
league_id = leagues_df['id'].to_numpy()

for season_name in season_names:
    seasons, has_more = run_api(f"https://api.sportmonks.com/v3/football/seasons/search/{season_name}?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep", True)
    
    page = 2
    while has_more == True:
        next_seasons, has_more = run_api(f"https://api.sportmonks.com/v3/football/seasons/search/{season_name}?page={page}&api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep", True)
        seasons['data'] = seasons['data'] + next_seasons['data']
        page += 1
    
    for season in seasons['data']:
        if season['league_id'] in league_id:
            season_data['data'].append(season)

for season in season_data['data']:
    if season['league_id'] in league_id:
        print(season)

{'id': 19734, 'sport_id': 1, 'league_id': 8, 'tie_breaker_rule_id': 1526, 'name': '2022/2023', 'finished': True, 'pending': False, 'is_current': False, 'starting_at': '2022-08-05', 'ending_at': '2023-05-28', 'standings_recalculated_at': '2023-11-30 15:26:55', 'games_in_current_week': False}
{'id': 19744, 'sport_id': 1, 'league_id': 82, 'tie_breaker_rule_id': 171, 'name': '2022/2023', 'finished': True, 'pending': False, 'is_current': False, 'starting_at': '2022-08-05', 'ending_at': '2023-05-27', 'standings_recalculated_at': '2023-11-30 15:44:34', 'games_in_current_week': False}
{'id': 19799, 'sport_id': 1, 'league_id': 564, 'tie_breaker_rule_id': 170, 'name': '2022/2023', 'finished': True, 'pending': False, 'is_current': False, 'starting_at': '2022-08-12', 'ending_at': '2023-06-04', 'standings_recalculated_at': '2023-08-09 14:53:21', 'games_in_current_week': False}
{'id': 19806, 'sport_id': 1, 'league_id': 384, 'tie_breaker_rule_id': 170, 'name': '2022/2023', 'finished': True, 'pending'

In [22]:
seasonByLeagueID = pd.DataFrame(season_data['data'])
seasonByLeagueID

,id,sport_id,league_id,tie_breaker_rule_id,name,finished,pending,is_current,starting_at,ending_at,standings_recalculated_at,games_in_current_week
0,19734,1,8,1526,2022/2023,True,False,False,2022-08-05,2023-05-28,2023-11-30 15:26:55,False
1,19744,1,82,171,2022/2023,True,False,False,2022-08-05,2023-05-27,2023-11-30 15:44:34,False
2,19799,1,564,170,2022/2023,True,False,False,2022-08-12,2023-06-04,2023-08-09 14:53:21,False
3,19806,1,384,170,2022/2023,True,False,False,2022-08-13,2023-06-11,2023-11-30 15:27:30,False
4,21207,1,648,577,2023,True,False,False,2023-04-15,2023-12-07,2023-12-07 02:41:01,False
5,21646,1,8,171,2023/2024,True,False,False,2023-08-11,2024-05-19,2024-05-19 17:01:55,False
6,21694,1,564,170,2023/2024,True,False,False,2023-08-11,2024-05-26,2024-05-26 21:06:57,False
7,21795,1,82,1526,2023/2024,True,False,False,2023-08-18,2024-05-18,2024-05-18 15:36:03,False
8,21818,1,384,170,2023/2024,True,False,False,2023-08-19,2024-06-02,2024-06-03 00:06:31,False


Get Teams by Season ID

In [39]:
season_id = seasonByLeagueID['id'].to_numpy()
teams_data = {'data':[]}

for id in season_id:
    teams, has_more = run_api(f"https://api.sportmonks.com/v3/football/teams/seasons/{id}?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep", False)
    print(f'Season id: {id}')
    
    page = 2
    while has_more == True:
        next_teams, has_more = run_api(f"https://api.sportmonks.com/v3/football/teams/seasons/{id}?page={page}&api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep")
        teams['data'] = teams['data'] + next_teams['data']
        page += 1
    
    for team in teams['data']:
        team['season_id'] = id
        teams_data['data'].append(team)

Season id: 19734
Season id: 19744
Season id: 19799
Season id: 19806
Season id: 21207
Season id: 21646
Season id: 21694
Season id: 21795
Season id: 21818


In [40]:
teams_df = pd.DataFrame(teams_data['data'])
teams_df

,id,sport_id,country_id,venue_id,gender,name,short_code,image_path,founded,type,placeholder,last_played_at,season_id
0,78,1,462,480,male,Brighton & Hove Albion,BHA,https://cdn.sportmonks.com/images/soccer/teams...,1901,domestic,False,2024-10-06 15:30:00,19734
1,11,1,462,485,male,Fulham,FUL,https://cdn.sportmonks.com/images/soccer/teams...,1879,domestic,False,2024-10-05 14:00:00,19734
2,8,1,462,230,male,Liverpool,LIV,https://cdn.sportmonks.com/images/soccer/teams...,1892,domestic,False,2024-10-05 11:30:00,19734
3,19,1,462,204,male,Arsenal,ARS,https://cdn.sportmonks.com/images/soccer/teams...,1886,domestic,False,2024-10-05 14:00:00,19734
4,42,1,462,117,male,Leicester City,LEI,https://cdn.sportmonks.com/images/soccer/teams...,1884,domestic,False,2024-10-05 14:00:00,19734
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,2930,1,251,1721,male,Inter,INT,https://cdn.sportmonks.com/images/soccer/teams...,1908,domestic,False,2024-10-05 18:45:00,21818
172,397,1,251,7220,male,Empoli,EMP,https://cdn.sportmonks.com/images/soccer/teams...,1920,domestic,False,2024-10-06 13:00:00,21818
173,1123,1,251,443,male,Hellas Verona,VER,https://cdn.sportmonks.com/images/soccer/teams...,1903,domestic,False,2024-10-04 18:45:00,21818
174,625,1,251,118500,male,Juventus,JUV,https://cdn.sportmonks.com/images/soccer/teams...,1897,domestic,False,2024-10-06 10:30:00,21818
